In [2]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 110.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.4 MB/s eta 0:00:00


In [3]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request


In [4]:
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)



In [5]:
# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary
task='emotion'

MODEL = f"cardiffnlp/twitter-roberta-base-{task}"
tokenizer = AutoTokenizer.from_pretrained(MODEL)


In [6]:
# download label mapping
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]



In [7]:
labels

['anger', 'joy', 'optimism', 'sadness']

In [8]:
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

In [9]:
text = "Celebrating my promotion 😎"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
scores

array([0.01446437, 0.93821114, 0.03615635, 0.01116814], dtype=float32)

In [10]:
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

1) joy 0.9382
2) optimism 0.0362
3) anger 0.0145
4) sadness 0.0112


In [11]:
def EmojiSA(text) :
    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    for i in range(scores.shape[0]):
        l = labels[ranking[i]]
        s = scores[ranking[i]]
        print(f"{i+1}) {l} {np.round(float(s), 4)}")

In [12]:
EmojiSA('I`m sad  😖  😖  😖 ')

1) sadness 0.9842
2) joy 0.0073
3) anger 0.005
4) optimism 0.0035


In [13]:
EmojiSA('I`m happy  😖  😖  😖 ')

1) joy 0.7872
2) sadness 0.183
3) optimism 0.0203
4) anger 0.0096


In [14]:
EmojiSA('I got my result  😖  😖  😖 ')

1) sadness 0.9238
2) anger 0.0557
3) joy 0.0141
4) optimism 0.0064


In [15]:
 EmojiSA('I got my result  🥳🥳🥳')

1) joy 0.9479
2) optimism 0.026
3) sadness 0.0131
4) anger 0.0129


In [16]:
 EmojiSA('I got my result  🤬🤬🤬')

1) anger 0.6644
2) sadness 0.16
3) joy 0.1586
4) optimism 0.0171


In [17]:
 EmojiSA('I got my result   🤪  🤪  🤪 ')

1) joy 0.9488
2) optimism 0.0254
3) anger 0.016
4) sadness 0.0098
